In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras import backend as K
import numpy as np
from tqdm import tqdm
from knn import kNN, convert_embeddings_to_tf
from load_mini_imagenet import load_mini_imagenet_data, split_for_n_shot
tf.compat.v1.enable_eager_execution()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def fn_loss(margin=1):
    """Provides 'triplet loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'triplet_loss' function with data ('margin') attached.
  """
    # L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    def triplet_loss(y_true, y_pred):
        anchor, positive, negative = y_pred[:,:emb_size], y_pred[:,emb_size:2*emb_size], y_pred[:,2*emb_size:]
        positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
      
        return tf.reduce_mean(tf.maximum(positive_dist - negative_dist + margin, 0.))
    

    return triplet_loss

In [4]:
model = tf.keras.models.load_model("weights/weights_mobilenet_pretrained_triplet_2.h5", custom_objects={'triplet_loss':fn_loss(1)})
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                            

In [5]:
X_test, y_test = load_mini_imagenet_data("mini-imagenet-cache-test.pkl")
X_support, y_support, X_test, y_test = split_for_n_shot(X_test, y_test, n=5)
mean = np.mean(X_support, axis=(0, 1, 2))
stddev = np.std(X_support, axis=(0, 1, 2))
X_support_norm = np.float32((X_support - mean) / stddev)
X_test_norm = np.float32((X_test - mean) / stddev)

In [6]:
support_features = model.layers[3](tf.image.resize(X_support_norm, (224, 224)))
support_features = layers.GlobalAveragePooling2D()(support_features).numpy()

In [7]:
support_features.shape

(100, 1280)

In [10]:
correct = 0
for i, label in tqdm(enumerate(y_test)):
    feat = model.layers[3](tf.image.resize(np.array([X_test_norm[i]]), (224, 224)))
    feat = layers.GlobalAveragePooling2D()(feat).numpy().squeeze()
    predicted = kNN(support_features, y_support, feat, k=1)
    if predicted == label:
        correct += 1

11900it [15:55, 12.45it/s]


In [11]:
print(correct / len(y_test))

0.6564705882352941


In [12]:
tf.compat.v1.disable_eager_execution()
convert_embeddings_to_tf(support_features, y_support, log_dir="logs/pretrained_mobilenet_triplet/")